In [1]:
import os
import pickle
import h5py
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy.stats import multivariate_normal
import math
import warnings
warnings.filterwarnings("ignore")


In [2]:
paper_params = {
    "d": 4,
    "m": 5,
    "neighbors_count": 2,
    "apply_pca": True,
    "pca_explained_variance_threshold": 0.95,
    "pca_min_components": 2,
    "gmm": {
        "cov_type": "full",
        "min_components": 1,
        "max_components": 12,
        "random_state": 0,
        "reg_covar": 1e-6
    },
    "cem": {
        "init_components": 8,
        "max_components": 12,
        "min_weight_threshold": 1e-3,
        "max_iter": 100,
        "tol": 1e-4,
        "reg_covar": 1e-6,
        "random_state": 0,
        "split_small": True,
    },
    "n_repeats": 5,
    "train_fraction": 2112/2400.0,
    "verbose": True,
    "center_sensor": None,
}


In [3]:
def load_pems_bay(h5_path="pems-bay.h5", adj_path="adj_mx_bay.pkl"):
    """
    Loads the PeMS-Bay dataset: speed matrix from an HDF5 file and 
    adjacency matrix from a pickle file. Supports multiple pickle formats.
    Ensures the adjacency matrix is extracted correctly.
    Returns speed_matrix and adjacency.
    """
    with h5py.File(h5_path, "r") as f:
        if 'speed' not in f:
            raise ValueError("'speed' key not found in h5 file")
        data = np.array(f['speed']['block0_values'])

    with open(adj_path, "rb") as f:
        adj_obj = pickle.load(f, encoding='latin1')

    adjacency = None
    if isinstance(adj_obj, dict):
        for v in adj_obj.values():
            if isinstance(v, np.ndarray) and v.ndim == 2:
                adjacency = v
                break
    elif isinstance(adj_obj, (list, tuple)):
        for v in adj_obj:
            if isinstance(v, np.ndarray) and v.ndim == 2:
                adjacency = v
                break
    elif isinstance(adj_obj, np.ndarray) and adj_obj.ndim == 2:
        adjacency = adj_obj

    if adjacency is None:
        raise ValueError("Could not parse adjacency matrix from pickle")

    return data, adjacency


In [4]:
def pick_sensor_neighborhood(adjacency, center_sensor=None, n_neighbors=2):
    """
    Selects a target sensor and its top-N strongest neighboring sensors 
    based on adjacency matrix weights. If center_sensor is not provided, 
    the middle sensor is chosen. Returns selected sensor IDs.
    """
    if center_sensor is None:
        center_sensor = adjacency.shape[0] // 2

    row = adjacency[center_sensor]
    neigh_idx = np.argsort(row)[::-1]
    neigh_idx = [i for i in neigh_idx if i != center_sensor]
    selected_neighbors = neigh_idx[:n_neighbors]
    sensor_ids = sorted(list(set(selected_neighbors + [center_sensor])))

    return sensor_ids, center_sensor


In [5]:
def build_cause_effect_matrices(speed_matrix, sensor_ids, d=4, m=5, target_sensor=None):
    """
    Creates the cause-effect dataset for time-series modeling.
    Uses lagged values of the target sensor (cause) and neighbor sensors (effect).
    Returns X (features), y (targets), timestamps, and the target sensor ID.
    """
    T, N = speed_matrix.shape
    if target_sensor is None:
        target_sensor = sensor_ids[len(sensor_ids)//2]

    neighbors = [s for s in sensor_ids if s != target_sensor]
    rows, ys, times = [], [], []
    start = max(d, m)

    for t in range(start, T):
        vec = []
        for lag in range(1, d + 1):
            vec.append(speed_matrix[t - lag, target_sensor])
        for nb in neighbors:
            for lag in range(1, m + 1):
                vec.append(speed_matrix[t - lag, nb])
        rows.append(vec)
        ys.append(speed_matrix[t, target_sensor])
        times.append(t)

    return np.asarray(rows), np.asarray(ys).reshape(-1, 1), times, target_sensor


In [6]:
def fit_pca_on_cause(X_train_cause, explained_variance_threshold=0.95, min_components=2, random_state=0):
    """
    Fits PCA on the cause part (lagged inputs). Automatically selects the 
    smallest number of components that preserve a required variance threshold. 
    Returns trained PCA, selected k components, and cumulative variance.
    """
    pca_full = PCA(n_components=min(X_train_cause.shape[1], 50),
                   svd_solver='full', random_state=random_state)
    pca_full.fit(X_train_cause)

    cum_var = np.cumsum(pca_full.explained_variance_ratio_)
    k = np.searchsorted(cum_var, explained_variance_threshold) + 1
    k = max(k, min_components)
    k = min(k, X_train_cause.shape[1])

    pca = PCA(n_components=k, svd_solver='full', random_state=random_state)
    pca.fit(X_train_cause)
    return pca, k, cum_var


In [7]:
def fit_gmm_bic(joint_train, min_k=1, max_k=12, cov_type='full',
                random_state=0, reg_covar=1e-6, verbose=False):
    """
    Trains GMM models with different component sizes and evaluates each 
    using Bayesian Information Criterion (BIC). Selects the model with 
    the lowest BIC score. Returns the best GMM model.
    """
    best_bic = np.inf
    best_gmm = None

    for k in range(min_k, max_k + 1):
        gmm = GaussianMixture(n_components=k, covariance_type=cov_type,
                              random_state=random_state, reg_covar=reg_covar)
        gmm.fit(joint_train)
        bic = gmm.bic(joint_train)

        if verbose:
            print(f"GMM k={k} BIC={bic}")

        if bic < best_bic:
            best_bic = bic
            best_gmm = gmm

    return best_gmm


In [8]:
def fit_cem_hard_em(joint_train, init_components=8, max_components=12, max_iter=100,
                    tol=1e-4, reg_covar=1e-6, random_state=0,
                    min_weight_threshold=1e-3, verbose=False, split_small=True):
    """
    Implements the Classification EM (CEM) algorithm for mixture modeling.
    Performs hard assignment of points to clusters, updates parameters,
    removes tiny components, and optionally splits dominant ones.
    Returns a dictionary containing weights, means, covariances, and component count.
    """
    rng = np.random.RandomState(random_state)
    n_samples, D = joint_train.shape
    K = init_components

    try:
        gmm0 = GaussianMixture(
            n_components=K, covariance_type='full',
            random_state=random_state, reg_covar=reg_covar
        )
        gmm0.fit(joint_train)
        weights = gmm0.weights_.copy()
        means = gmm0.means_.copy()
        covs = gmm0.covariances_.copy()
    except Exception:
        indices = rng.choice(n_samples, size=K, replace=False)
        means = joint_train[indices].copy()
        covs = np.array([
            np.cov(joint_train.T) + reg_covar * np.eye(D)
            for _ in range(K)
        ])
        weights = np.ones(K) / K

    prev_assign = None

    for it in range(max_iter):

        resp = np.zeros((n_samples, K))
        for k in range(K):
            try:
                resp[:, k] = weights[k] * multivariate_normal.pdf(
                    joint_train, mean=means[k], cov=covs[k], allow_singular=True
                )
            except:
                resp[:, k] = weights[k] * 1e-300

        s = resp.sum(axis=1, keepdims=True)
        z = np.where(s > 0, resp / s, np.ones_like(resp) / K)

        assigns = np.argmax(z, axis=1)

        if prev_assign is not None and np.array_equal(assigns, prev_assign):
            if verbose:
                print(f"CEM converged at iteration {it}")
            break

        prev_assign = assigns.copy()

        new_means, new_covs, new_weights = [], [], []
        unique, counts = np.unique(assigns, return_counts=True)

        for k in range(K):
            members = joint_train[assigns == k]
            nk = members.shape[0]

            if nk == 0:
                new_means.append(means[k])
                new_covs.append(covs[k])
                new_weights.append(0.0)
            else:
                mu_k = members.mean(axis=0)
                if nk > 1:
                    cov_k = np.cov(members.T) + reg_covar * np.eye(D)
                else:
                    cov_k = np.eye(D) * reg_covar

                new_means.append(mu_k)
                new_covs.append(cov_k)
                new_weights.append(nk / n_samples)

        new_means = np.vstack(new_means)
        new_covs = np.stack(new_covs)
        new_weights = np.array(new_weights)

        keep_idx = new_weights >= min_weight_threshold
        if not np.any(keep_idx):
            keep_idx[np.argmax(new_weights)] = True

        means = new_means[keep_idx]
        covs = new_covs[keep_idx]
        weights = new_weights[keep_idx]

        weights /= weights.sum()
        K = means.shape[0]

        if split_small and K < max_components:
            big_idx = np.argmax(weights)

            if weights[big_idx] > 2.0 / K:
                mu_big = means[big_idx]
                cov_big = covs[big_idx]

                perturb = rng.normal(
                    scale=np.sqrt(np.diag(cov_big)) * 0.01, size=mu_big.shape
                )

                mu1 = mu_big + perturb
                mu2 = mu_big - perturb

                w_big = weights[big_idx] / 2.0

                means = np.delete(means, big_idx, axis=0)
                covs = np.delete(covs, big_idx, axis=0)
                weights = np.delete(weights, big_idx, axis=0)

                means = np.vstack([means, mu1, mu2])
                covs = np.concatenate([covs, cov_big[None], cov_big[None]])
                weights = np.concatenate([weights, np.array([w_big, w_big])])

                weights /= weights.sum()
                K = means.shape[0]

        if verbose:
            print(f"Iteration {it}: Components = {K}")

    return {
        "weights": weights,
        "means": means,
        "covariances": covs,
        "n_components": means.shape[0]
    }


In [9]:
def predict_mmse_from_gmm_like(model_like, X_cause_reduced,
                               dim_effect=1, reg_covar=1e-8):
    """
    Generates MMSE predictions from either a trained GMM model or a CEM output.
    Computes conditional mean of effect variables given cause variables.
    Returns predicted future values for the target sensor.
    """
    if isinstance(model_like, GaussianMixture):
        weights = model_like.weights_
        means = model_like.means_
        covs = model_like.covariances_
    else:
        weights = model_like["weights"]
        means = model_like["means"]
        covs = model_like["covariances"]

    K = len(weights)
    D = means.shape[1]
    dE = X_cause_reduced.shape[1]
    dF = dim_effect

    preds = np.zeros((X_cause_reduced.shape[0], dF))

    for i, xE in enumerate(X_cause_reduced):
        comp_means = []
        comp_weights = []

        for l in range(K):
            mu = means[l]
            Sigma = covs[l]

            mu_E = mu[:dE]
            mu_F = mu[dE:]

            Sigma_EE = Sigma[:dE, :dE] + reg_covar * np.eye(dE)
            Sigma_FE = Sigma[dE:, :dE]

            inv_EE = np.linalg.pinv(Sigma_EE)
            mu_F_given_E = mu_F + Sigma_FE.dot(inv_EE).dot(xE - mu_E)

            try:
                p_xE_l = multivariate_normal.pdf(
                    xE, mean=mu_E, cov=Sigma_EE, allow_singular=True
                )
            except:
                p_xE_l = 1e-300

            comp_means.append(mu_F_given_E)
            comp_weights.append(weights[l] * p_xE_l)

        comp_weights = np.array(comp_weights)
        if comp_weights.sum() == 0:
            comp_weights = np.ones_like(comp_weights) / len(comp_weights)
        else:
            comp_weights /= comp_weights.sum()

        preds[i] = np.sum(
            [w * m for w, m in zip(comp_weights, comp_means)], axis=0
        )

    return preds


In [11]:
def run_pipeline(speed_matrix, adjacency, center_sensor=None,
                 params=paper_params, method="gmm_bic"):
    """
    Runs the complete workflow: selects sensors, builds cause-effect matrices,
    performs PCA reduction, fits GMM or CEM, and evaluates RMSE over repeats.
    Returns performance metrics and selected sensor information.
    """
    sensor_ids, center = pick_sensor_neighborhood(
        adjacency, center_sensor, params["neighbors_count"]
    )
    print("Sensors:", sensor_ids, "| Target:", center)

    X_cause_all, y_all, times, target = build_cause_effect_matrices(
        speed_matrix, sensor_ids, params["d"], params["m"], center
    )

    n_samples = X_cause_all.shape[0]
    train_frac = params["train_fraction"]
    n_repeats = params["n_repeats"]

    rng = np.random.RandomState(params["gmm"]["random_state"])
    rmse_list = []

    for rep in range(n_repeats):
        idx = np.arange(n_samples)
        rng.shuffle(idx)
        n_train = int(train_frac * n_samples)

        train_idx = idx[:n_train]
        test_idx  = idx[n_train:]

        X_train_cause = X_cause_all[train_idx]
        y_train       = y_all[train_idx]
        X_test_cause  = X_cause_all[test_idx]
        y_test        = y_all[test_idx]

        if params["apply_pca"]:
            pca, k, cumvar = fit_pca_on_cause(
                X_train_cause,
                params["pca_explained_variance_threshold"],
                params["pca_min_components"]
            )
            X_train_red = pca.transform(X_train_cause)
            X_test_red  = pca.transform(X_test_cause)
        else:
            X_train_red = X_train_cause
            X_test_red  = X_test_cause

        joint_train = np.hstack([X_train_red, y_train])

        if method == "gmm_bic":
            model = fit_gmm_bic(
                joint_train,
                params["gmm"]["min_components"],
                params["gmm"]["max_components"],
                params["gmm"]["cov_type"],
                params["gmm"]["random_state"] + rep,
                params["gmm"]["reg_covar"],
                params["verbose"]
            )
        else:
            model = fit_cem_hard_em(
                joint_train,
                params["cem"]["init_components"],
                params["cem"]["max_components"],
                params["cem"]["max_iter"],
                params["cem"]["tol"],
                params["cem"]["reg_covar"],
                params["cem"]["random_state"] + rep,
                params["cem"]["min_weight_threshold"],
                params["verbose"],
                params["cem"]["split_small"]
            )

        preds = predict_mmse_from_gmm_like(
            model, X_test_red, dim_effect=1
        )

        rmse = math.sqrt(mean_squared_error(y_test.ravel(), preds.ravel()))
        rmse_list.append(rmse)

        print(f"Repeat {rep+1}/{n_repeats}: RMSE = {rmse:.4f}")

    print("\nFinal Mean RMSE:", np.mean(rmse_list))
    print("Final Std Dev RMSE:", np.std(rmse_list))

    return {
        "sensor_ids": sensor_ids,
        "target_sensor": target,
        "rmse_list": rmse_list,
        "rmse_mean": float(np.mean(rmse_list)),
        "rmse_std": float(np.std(rmse_list))
    }


In [12]:
speed_matrix, adjacency = load_pems_bay("pems-bay.h5", "adj_mx_bay.pkl")

paper_params["center_sensor"] = 100

print("\nRunning GMM-BIC Pipeline:")
gmm_results = run_pipeline(speed_matrix, adjacency, method="gmm_bic")

print("\nRunning CEM Pipeline:")
cem_results = run_pipeline(speed_matrix, adjacency, method="cem")



Running GMM-BIC Pipeline:
Sensors: [162, 163, 242] | Target: 162
GMM k=1 BIC=1048914.7206562506
GMM k=2 BIC=863330.9728658521
GMM k=3 BIC=810796.2458046193
GMM k=4 BIC=799941.1450278114
GMM k=5 BIC=788659.1476963651
GMM k=6 BIC=787189.0096543011
GMM k=7 BIC=783616.581653082
GMM k=8 BIC=776611.3326357132
GMM k=9 BIC=776883.7105246867
GMM k=10 BIC=772904.1574328252
GMM k=11 BIC=772555.4662571248
GMM k=12 BIC=771374.8347530906
Repeat 1/5: RMSE = 2.7529
GMM k=1 BIC=1049048.7581357763
GMM k=2 BIC=863364.1068554256
GMM k=3 BIC=811144.5835214859
GMM k=4 BIC=800366.815651246
GMM k=5 BIC=798538.11259822
GMM k=6 BIC=787807.2443001752
GMM k=7 BIC=780342.1283501382
GMM k=8 BIC=778736.5406757437
GMM k=9 BIC=775582.0550543846
GMM k=10 BIC=774836.4901657586
GMM k=11 BIC=772401.420813902
GMM k=12 BIC=772945.323965509
Repeat 2/5: RMSE = 2.7438
GMM k=1 BIC=1048508.6117270412
GMM k=2 BIC=862766.6866269008
GMM k=3 BIC=810225.7245832399
GMM k=4 BIC=799449.3108169233
GMM k=5 BIC=797630.8968359494
GMM k=6 B